### 决策树


In [36]:
import numpy as np
# 初分类 对单独列进行元素和label的分类
# 数据 西瓜书 第76页
x = np.array(([[1,1,1,1,1,1],
               [2,1,2,1,1,1],
               [2,1,1,1,1,1],
               [1,1,2,1,1,1],
               [3,1,1,1,1,1],
               [1,2,1,1,2,2],
               [2,2,1,2,2,2],
               [2,2,1,1,2,1],
               [2,2,2,2,2,1],
               [1,3,3,1,3,2],
               [3,3,3,3,3,1],
               [3,1,1,3,3,2],
               [1,2,1,2,1,1],
               [3,2,2,2,1,1],
               [2,2,1,1,2,2],
               [3,1,1,3,3,1],
               [1,1,2,2,2,1]]))
y = np.array(([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0]))

In [37]:
#获得y中每种label的个数
def get_lable_count(y):
    all_lable_count = {}
    for lable in y:
        if lable in all_lable_count.keys():
            all_lable_count[lable] = all_lable_count[lable] + 1
        else:
            all_lable_count[lable] = 1
    return all_lable_count

在上面的函数中，使用get_lable_count函数，可以对训练数据中不同的lable计数，获得每个lable对应的训练数据的个数。

计算$Ent(D)=- \sum_\limits{i=1}^{n}p_i log p_i$

In [38]:
import math
all_lable_count = get_lable_count(y)
counts = len(y)
# 计算数据整体的entropy
def calcu_entropy(all_lable_count, counts):
    lables = all_lable_count.keys()
    sum = 0
    for lable in lables:
        rate = all_lable_count[lable]/counts
        sum += rate * math.log2(rate)  
    return -sum


In [39]:
# 对第column列进行数据划分
# 返回一个字典，字典{factor:[{label_1,count},{lable_2, count}]
def divide_data(x, y, column):
    data = np.column_stack((y, x)) # 对数据进行结合，方便遍历数据
    dict_factor_lable_count = {}   # 字典类型，key为第column列的属性值，value为 label和lable count的数组。
    dict_index = {} # 字典类型，第column列中，相同factor的index
    count = 0
    for row_x_y in data:
        factor_key = dict_factor_lable_count.keys()
        factor = row_x_y[1+column] # 第column列元素
        lable = row_x_y[0]         # lable
        if( factor in factor_key):
            dict_index[factor].append(count)
            count += 1
            lable_count_array = dict_factor_lable_count[factor]
            for index in range(0,len(lable_count_array)):
                if lable in lable_count_array[index].keys():
                    lable_count_array[index][lable] = lable_count_array[index][lable] + 1
                    break
                elif index + 1 == len(lable_count_array):
                    tmp_dict = {lable: 1}
                    lable_count_array.append(tmp_dict)
                    break
        else:
            dict_factor_lable_count[factor] = [{lable: 1}]
            dict_index[factor] = [count]
            count +=1
    return (dict_factor_lable_count, dict_index) 
    

# ({1: [{1: 3}, {0: 1}], 2: [{0: 3}], 3: [{0: 3}, {1: 2}], 5: [{1: 1}]},
#  {1: [{0}, {5}, {7}, {11}], 2: [{1}, {10}, {12}], 3: [{2}, {3}, {4}, {8}, {9}], 5: [{6}]})
#  对x 的第0列进行划分，可以得到，当第0列元素是1时，lable=1的样本有3个，lable=0的样本有1个，依此类推。
#  数据按着第column列进行划分，得到数据的index。

通过上面的函数divide_data，我们可以对第colume列进行划分，使得第column列元素相同的放在一起，同时再将第column列相同的元素按着lable进行计数

计算$Ent(D^v) = - \sum_\limits{i=1}^{n}p_i log p_i$

In [40]:
(dict_factor_lable_count, dict_index) = divide_data(x,y,0)
# 计算信息熵
import math
def calcu_part_entropy(dict_factor_lable_count, dict_index):
    factor_set = dict_factor_lable_count.keys()
    list_entropy = []
    count = 0
    for factor in factor_set:
        lable_count_array=dict_factor_lable_count[factor]
        factor_all_count = len(dict_index[factor])
        sum = 0
        for lable_count in lable_count_array:
            for lable in lable_count.keys():
                rate = lable_count[lable] / factor_all_count
                sum += rate*math.log(rate,2)
        list_entropy.append({factor: -sum})

    return list_entropy

计算信息增益，是用第column个元素对数据集进行划分，就会产生$v$个样本集合，再考虑到每个样本集合包含的样本数不同，对不同的样本赋予权重$\frac{D^v}{D}$，然后计算第column个元素对数据进行划分所得到的信息增益。
$$Gain(D,factor)=Ent(D)- \sum \limits_{i=1}^{v}\frac{D^v}{D} Ent(D^v)$$
根据上面计算出的第column列元素的信息熵，计算第column的信息增益。

In [41]:
# 计算信息增益
def calcu_gain(counts, D_entropy, D_v_entropy, dict_index):
    sum = 0
    for factor_entropy in D_v_entropy:
        factors = factor_entropy.keys()
        for factor in factors:
            sum +=(len(dict_index[factor])/counts) * factor_entropy[factor]
    return D_entropy - sum

In [42]:
counts = len(y)
D_entropy = calcu_entropy(get_lable_count(y),counts)
for index in range(0, len(x[0])):
    dict_factor_lable_count, dict_index = divide_data(x,y,index)
    D_v_entropy = calcu_part_entropy(dict_factor_lable_count, dict_index)
    print(calcu_gain(counts, D_entropy, D_v_entropy, dict_index))

0.10812516526536531
0.14267495956679288
0.14078143361499584
0.3805918973682686
0.28915878284167895
0.006046489176565584


In [43]:
# 用于构建决策树的类
class Node:
    factor_index = -1
    list_node = None #数组中的数据类型为 node
    list_factors = None # 数组中的数据类型为当前节点判断的元素是哪个，factor的index与 list_node中对应的node的index一样
    is_lable = False
    lable = None
    
    def __init__(self, factor_index, list_node, list_factors, is_lable, lable):
        self.factor_index = factor_index
        self.list_node = list_node
        self.list_factors = list_factors
        self.is_lable = is_lable
        if is_lable:
            self.lable = lable
    def __init__(self):
        self.list_node = []
        self.list_factors = []
        pass

In [62]:
# 
def decision_factor(x, y, factor_set):
    counts = len(y)
    D_entropy = calcu_entropy(get_lable_count(y),counts)
    
    order = -1
    dict_gain_index = {"gain":-1, "index":-1}
    dict_factor_index = None
    
    for index in range(0, len(x[0])):
        if index in factor_set:
            continue
        else:
            dict_factor_lable_count, dict_index = divide_data(x,y,index)
            D_v_entropy = calcu_part_entropy(dict_factor_lable_count, dict_index)
            gain = calcu_gain(counts, D_entropy, D_v_entropy, dict_index)
            print(gain)
            if gain > dict_gain_index["gain"]:
                dict_gain_index["gain"]= gain
                dict_gain_index["index"] = index
                dict_factor_index = dict_index
    factor_set.add(dict_gain_index["index"])
    return (dict_gain_index, dict_factor_index)

In [45]:
import numpy as np
import copy
def build_without_tree(x,y,factor_set):
    name = ["色泽","根蒂","敲声","纹理","肚部","触感"]
    dict_gain_index,dict_factor_lable_count = decision_factor(x, y, factor_set)
    factors = dict_factor_lable_count.keys()
   # print(factor_set)
    tmp_set = copy.deepcopy(factor_set)
    for factor in factors:
        count = 0
        indexs = dict_factor_lable_count[factor]
        tmp_x = []
        tmp_y = []
        lable_set = set()
        for index in indexs:
            lable_set.add(y[index])
            tmp_y.append(y[index])
            tmp_x.append(x[index])
        
        if len(lable_set)==1:
            pass
        else:
            build_without_tree(tmp_x,tmp_y,factor_set)
    pass

factor_set = set()
node = Node()
build_without_tree(x,y, factor_set)

In [46]:
import numpy as np
import copy
def build_tree(x, y, factor_set, node):
    name = ["色泽","根蒂","敲声","纹理","肚部","触感"]
    dict_gain_index,dict_factor_lable_count = decision_factor(x, y, factor_set)
    factors = dict_factor_lable_count.keys()
    node.factor_index=dict_gain_index["index"]
    tmp_set = copy.deepcopy(factor_set)
    for factor in factors:
        count = 0
        indexs = dict_factor_lable_count[factor]
        tmp_x = []
        tmp_y = []
        lable_set = set()
        for index in indexs:
            lable_set.add(y[index])
            tmp_y.append(y[index])
            tmp_x.append(x[index])
        
        if len(lable_set)==1:
            tmp_node = Node()
            tmp_node.factor_index = factor
            tmp_node.is_lable = True
            tmp_node.lable = list(lable_set)[0]
            node.list_node.append(tmp_node)
            node.list_factors.append(factor)
            pass
        else:
            tmp_node = Node()
            node.list_factors.append(factor)
            node.list_node.append(tmp_node)
            build_tree(tmp_x,tmp_y,tmp_set, tmp_node)
            pass

In [63]:
factor_set = set()
node = Node()
build_tree(x, y, factor_set, node)
node

0.10812516526536531
0.14267495956679288
0.14078143361499584
0.3805918973682686
0.28915878284167895
0.006046489176565584
0.04306839587828004
0.45810589515712374
0.33085622540971754
0.45810589515712374
0.45810589515712374
0.2516291673878229
0.0
0.0
0.2516291673878229
0.0
0.0
1.0
0.3219280948873623
0.3219280948873623
0.17095059445466865
0.7219280948873623


In [60]:
node.list_node[0].factor_index

1

In [49]:
class DescisionTree:
    node = None
    
    def fit(self,x,y):
        factor_set = set()
        node = Node()
        build_tree(x, y, factor_set, node)
        self.node = node
    
    def predict(self,x):
        flag = True
        exception = False
        tmp_node = self.node
        result = None
        while flag:
            if tmp_node.is_lable:
                result = tmp_node.lable
                flag = False
            else:
                index = 0
                for factor in tmp_node.list_factors:
                    if factor == x[tmp_node.factor_index]:
                        tmp_node = tmp_node.list_node[index]
                        break
                    else:
                        pass
                    index += 1
                    if index == len(tmp_node.list_factors):
                        exception = True
                        flag = False
        
        return result

In [50]:
ds = DescisionTree()
ds.fit(x,y)

print(ds.predict([1,2,3,2,1,1]))

0


In [61]:
len(node.list_node[0].list_node[1].list_node)

2